In [1]:
import pandas as pd
import numpy as np 
import time
import os
import glob
import requests


## Notebook to test and compare different prompts with human read papers

In [11]:

def upload_analyze_papers(food, hazard, API_Key, question='default'):
    
    ########################## set up run ##########################
    # make an empty inital dataframe to fill in 
    all_results = pd.DataFrame(columns=['filename', 'food', 'hazard', 'gen_quote', 'impact', 'direction', 'mechanism', 'quote', 'location', 'full_answer']) 

    # specify dwn_dir (formatted: food_hazard)
    dwn_dir = os.path.join(os.getcwd(), f'{food}_{hazard}')
    # look for pds
    pdfs = glob.glob(os.path.join(dwn_dir, '*.pdf'))
    # make print to check what is going on
    print(f'analyzing {food} and {hazard}. Found {str(len(pdfs))} papers', flush=True, end=' ')
    
    
    ########################## Load papers into chatpdf ##########################
    
    ### dummy loop to only upload 12/minute (limit for free account - change if we have paid account)
    # uploads = 0
    # start_time = time.time()
    for file in pdfs:
    #     # enter only here if upload number is 12 or more
    #     if (uploads >= 12) & elapsed_time < 60:
    #         elapsed_time = time.time() - start_time
    #         print('uplaode 12',elapsed_time)

    #         if elapsed_time < 60:
    #             # If uploads exceed the limit in less than a minute, wait for the remaining time
    #             wait_time = 60 - elapsed_time
    #             print(f'Upload limit reached. Waiting for {wait_time:.2f} seconds...')
    #             time.sleep(wait_time)
    #             uploads = 0  # Reset the counter and start a new minute
    #             start_time = time.time()
        ###

        files = [('file', ('file', open(file, 'rb'), 'application/octet-stream'))]
        headers = {'x-api-key': API_Key}
        response = requests.post('https://api.chatpdf.com/v1/sources/add-file', headers=headers, files=files)
        # Update the upload counter
        #uploads = uploads + 1
        ## chatpdf jargon; uploading papers
        if response.status_code == 200:
            sID = response.json()['sourceId']
        else:
            print('Status:', response.status_code)
            print('Error:', response.text)
            continue


        ########################## 'Read' papers in chatpdf ##########################
        headers = {
            'x-api-key': API_Key,
            "Content-Type": "application/json"
        }
        # the defualt question isnt really working with current dataframe setup
        if question == 'default':
            question = f'provide a quote from the text about how {hazard} impacts {food}?'
        
        # more chatpdf jargon, input question:
        data = {
            'sourceId': sID,
            'messages': [
                # set up few-shot prompting (i'm not entirely sure if we should have our same questions before this to show it how to answer or if this is sufficient)
                # code from: https://www.chatpdf.com/docs/api/backend
                {
                    'role': "assistant",
                    'content': '1a) "Understanding the responses of animals to acute heat stress can help to reveal and predict the effect of more frequent extreme hot weather episodes on animal populations and ecosystems in the content of global climate change". 2a) Indirect physiology 3a) Negative 4a) The warming negatively impacts Chinese softshell turtle through indirect physiological mechanisms. 5a) "The results suggest that the Chinese soft-shelled turtle has a strong capacity to activate its antioxidant defense system to cope with acute heat stress". 6a) Beijing, China. 7a) The impact of warming on Chinese softshell turtle via indirect physiology is negative. The study was in Beijing, China.'
                },
                # actual question
                {
                    'role': "user",
                    'content': question,
                }
            ]
        }
        response = requests.post(
            'https://api.chatpdf.com/v1/chats/message', headers=headers, json=data)

        # if results are returned
        if response.status_code == 200:
            #print('result:', response.json()['content'])
            results = response.json()['content']
            segments = results.split('a)')  # Split the text using the question format ('a)')
            #segments = results.split('\n')  # could also split it by lines - might work well if a) is bad
            print(segments)
            print(len(segments))
            # split answer into small segments to put in dataframe
            ### segments for a. 
            if len(segments) ==8:
                gen_quote = segments[1].strip()[:-3]
                impact = segments[2].strip()[:-3]
                direction = segments[3].strip()[:-3]
                mechanism = segments[4].strip()[:-3]
                quote = segments[5].strip()[:-3]
                location = segments[6].strip()[:-3]
                full_answer = segments[7].strip()[:-3]

            else:
                gen_quote = 'Nan'
                impact = 'Nan'
                direction = 'Nan'
                mechanism = 'Nan'
                quote = 'Nan'
                location = 'Nan'
                full_answer = 'Nan'
            
            print(full_answer)
            # Create a row dictionary for this PDF
            row_data = {
                'filename': os.path.basename(file),
                'food': food,
                'hazard': hazard,
                'gen_quote': gen_quote,
                'impact': impact,
                'direction': direction,
                'mechanism': mechanism,
                'quote': quote,
                'location': location,
                'full_answer': full_answer
            }

            # Append the row data to the results
            all_results = pd.concat([all_results, pd.DataFrame([row_data])], ignore_index=True)

        else: #if it's baaad
            print('Status:', response.status_code)
            print('Error:', response.text)
    # Return dataframe for one pdf
    return all_results 

In [12]:
# BULK TEST with read papers

foods = ['Bay leaf','Bignose unicornfish','Button mushroom','Centropristis philadelphica','Chinese softshell turtle','Corica soborna','Ganges river sprat','Grasshopper','Large yellow croaker','Larimichthys croceus','Leucoraja erinacea','Little skate','mealworm','Naso vlamingii','Orange-spotted grouper','Ordways brotula','Pusa hispida','Ringed seal','Rock sea bass','Tenebrio molitor','Trionyx sinensis','Triticum aestivum','wheat']
hazards = ['atmospheric CO2 increases','ocean acidification','ozone','storms','warming','precipitation','precipitation','fires','warming','warming','ocean acidification','ocean acidification','warming','ocean acidification','natural cover change','floods','heatwaves','Heatwaves','storms','warming','warming','drought','drought']
API_Key = 'sec_uWPhoztZ5o1f4iiBRrudtvdWONGJKPp0' #make a user and get api key from https://www.chatpdf.com/docs/api/backend

# make an inital empty dataframe
dfall = pd.DataFrame(columns=['filename', 'food', 'hazard', 'gen_quote', 'impact', 'direction', 'mechanism', 'quote', 'location', 'full_answer']) 

# loop through food/hazard combo
count = 0
start_time = time.time()
for i in range(len(foods)):
    ### DUMMY TIMER 
    count += 1
    # enter only here if upload number is 12 or more
    if (count >= 12):
        elapsed_time = time.time() - start_time
        print('count is 12', round(elapsed_time,2))
        if elapsed_time < 60:
            # If uploads exceed the limit in less than a minute, wait for the remaining time
            wait_time = 60 - elapsed_time
            print(f'Upload limit reached. Waiting for {wait_time:.2f} seconds...')
            time.sleep(wait_time)
            count = 0  # Reset the counter and start a new minute
            start_time = time.time()

    ######### 
    food = foods[i]
    hazard = hazards[i]

    print(food, hazard)
    # make the question (food/hazard specific)
    question = (
        'please include a) after each question, like 1a), 7a) '
        f'if you cannot find proof that {hazard} impacts {food}, just reply No impact found and disregard the other questions. If you do find an impact, please answer the following: '
        f'1a) provide a quote from the text about how {hazard} impacts {food}? '
        f'2a) please classify the impact of {hazard} on {food} as direct physiology, indirect abiotic, indirect biotic. Respond only direct physiology, indirect abiotic, indirect biotic. If no effect is found respond no effect '
        f'3a) does {hazard} negatively, positively or neutrally impact {food} (Respond only with negatvie, positive, neutral, or No effect)'
        f'4a) If {food} is impacted by {hazard}, can you explain through what mechanisms? else, reply Na '
        f'5a) Please provide a quote from the text about the mechanism. if there is none reply Na '
        f'6a) select where this study took place. if its a lab this experment, select lab '
        f'7a) Provide an answer in the following format: The impact of {hazard} on {food} via (select: direct physiology, indirect abiotic, '
        f'or indirect biotic) are (select: positive, negative or neutral). The study was in (location). if no effect is found reply: no effect was found'
        )
    #ruuun
    df = upload_analyze_papers(foods[i], hazards[i], API_Key, question=question)
    # combine with master dataframe
    dfall = pd.concat([dfall, df], ignore_index=True)

dfall

Bay leaf atmospheric CO2 increases
analyzing Bay leaf and atmospheric CO2 increases. Found 1 papers 

['I apologize, but I cannot find any information about atmospheric CO2 increases and their impact on Bay leaf in the given pages.']
1
Nan
Bignose unicornfish ocean acidification
analyzing Bignose unicornfish and ocean acidification. Found 1 papers ['No impact found.']
1
Nan
Button mushroom ozone
analyzing Button mushroom and ozone. Found 1 papers ['1', ' No impact found on Button mushroom. \nTherefore, questions 2a-7a are not applicable.']
2
Nan
Centropristis philadelphica storms
analyzing Centropristis philadelphica and storms. Found 1 papers ['No impact found.']
1
Nan
Chinese softshell turtle warming
analyzing Chinese softshell turtle and warming. Found 1 papers ['1', ' "The results suggest that the Chinese soft-shelled turtle has a strong capacity to activate its antioxidant defense system to cope with acute heat stress."\n2', ' Indirect abiotic\n3', ' Negative\n4', ' The warming negatively impacts Chinese softshell turtle through indirect abiotic mechanisms.\n5', ' "The results sug

,filename,food,hazard,gen_quote,impact,direction,mechanism,quote,location,full_answer
0,Predicting of the current and future geographi...,Bay leaf,atmospheric CO2 increases,Nan,Nan,Nan,Nan,Nan,Nan,Nan
1,Physiological implications of ocean acidificat...,Bignose unicornfish,ocean acidification,Nan,Nan,Nan,Nan,Nan,Nan,Nan
2,"Effect of Applied Ozone Dose, Time of Ozonizat...",Button mushroom,ozone,Nan,Nan,Nan,Nan,Nan,Nan,Nan
3,Proximity Effects of Larger Resident Fishes on...,Centropristis philadelphica,storms,Nan,Nan,Nan,Nan,Nan,Nan,Nan
4,Response of the Chinese Soft-Shelled Turtle to...,Chinese softshell turtle,warming,"""The results suggest that the Chinese soft-she...",Indirect abioti,Negativ,The warming negatively impacts Chinese softshe...,"""The results suggest that the Chinese soft-she...","Beijing, China",The impact of warming on Chinese softshell tur...
5,Temporal distribution of fisheries in Payra Ri...,Corica soborna,precipitation,Nan,Nan,Nan,Nan,Nan,Nan,Nan
6,Temporal distribution of fisheries in Payra Ri...,Ganges river sprat,precipitation,Nan,Nan,Nan,Nan,Nan,Nan,Nan
7,Grasshopper (Orthoptera Acrididae) communities...,Grasshopper,fires,Nan,Nan,Nan,Nan,Nan,Nan,Nan
8,"Thermal tolerance, safety margins and acclimat...",Large yellow croaker,warming,"""Since marine heatwaves usually persist for da...",Indirect physiology,Negative,Large yellow croaker may suffer potentially hi...,"""if the SSTs during future heatwaves reach or ...","Xiamen and Ningde, China",The impact of warming on Large yellow croaker ...
9,"Thermal tolerance, safety margins and acclimat...",Larimichthys croceus,warming,Nan,Nan,Nan,Nan,Nan,Nan,Nan


In [92]:
### SAVE OUTPUT as CSV (or another format)
filename = 'output_ALL1.csv'
dfall.to_csv(filename)

In [52]:
#####  Single Test ######

food = 'Button mushroom'
hazard = 'ozone'
API_Key = 'sec_6LIDxgLBHBqmhkVam818PYYXqervcPSX' #make a user and get api key from https://www.chatpdf.com/docs/api/backend
#question  = f'1a) provide a quote from the text about how {hazard} impacts {food}? 2a) in what region is this study 3a) does {hazard} negatively or positively impact {food} (reply only negatvie/positive) 4a) would you classify the impact on {food} as direct physiology, indirect abiotic, or indirect biotic?  5a) can you give a quote providing an example of this?'

question = (
    f'1a) provide a quote from the text about how {hazard} impacts {food}? '
    f'2a) please classify the impact on {food} as direct physiology, indirect abiotic, indirect biotic, or none. Respond only direct physiology, indirect abiotic, indirect biotic, or no effect'
    f'3a) does {hazard} negatively, positively or neutrally impact {food} (Respond only with negatvie, positive, neutral, or No effect)'
    f'4a) If {food} is impacted by {hazard}, can you explain through what mechanisms? else, reply Na '
    f'5a) Please provide a quote from the text about the mechanism. if there is none reply Na '
    f'6a) select where this study took place. if its a lab this experment, select lab ' 
    f'7a) Provide an answer in the following format: The impact of {hazard} on {food} via (select: direct physiology, indirect abiotic, '
    f'or indirect biotic) are (select: positive, negative or neutral). The study was in (location). if no effect is found reply: no effect was found'
    )


all_results = upload_analyze_papers(food, hazard, API_Key, question=question)
all_results


found 1 paper(s)
loading file "Effect of Applied Ozone Dose, Time of Ozonization, and Storage Time on Selected Physicochemical Characteristics of Mushrooms (Agaricus bisporus).pdf" 
analyzing file...
['1', ' According to the text, "Exposure of mushrooms to ozone atmosphere does not cause large changes in quality and physical parameters of Agaricus bisporus."\n2', ' No effect.\n3', ' Neutral.\n4', ' Na.\n5', ' Na.\n6', ' The study was conducted in a lab.\n7', ' The impact of ozone on Button mushroom is no effect. The study was conducted in a lab.']
test1
test2
The impact of ozone on Button mushroom is no effect. The study was conducted in a lab.




To do for prompt test
- load papers
- get comporable output as human df
- make list/dict/df with question and dfID


To do for big tool
- find effective way to combine bibtxt csv and/or pdfs into groupings that make sense
- either append resutls here or on pypaperbot csv
- subset run to 'smaller' batches 


In [10]:
### Test csv as input to chatpdf


In [ ]:

def upload_analyze_papers(food, hazard, API_Key, question='default'):
    
    ########################## set up run ##########################
    # make an empty inital dataframe to fill in 
    all_results = pd.DataFrame(columns=['filename', 'food', 'hazard', 'gen_quote', 'impact', 'direction', 'mechanism', 'quote', 'location', 'full_answer']) 

    # specify dwn_dir (formatted: food_hazard)
    dwn_dir = os.path.join(os.getcwd(), f'{food}_{hazard}')
    # look for pds
    pdfs = glob.glob(os.path.join(dwn_dir, '*.pdf'))
    # make print to check what is going on
    print(f'analyzing {food} and {hazard}. Found {str(len(pdfs))} papers', flush=True, end=' ')
    
    
    ########################## Load papers into chatpdf ##########################
    
    ### dummy loop to only upload 12/minute (limit for free account - change if we have paid account)
    # uploads = 0
    # start_time = time.time()
    for file in pdfs:
    #     # enter only here if upload number is 12 or more
    #     if (uploads >= 12) & elapsed_time < 60:
    #         elapsed_time = time.time() - start_time
    #         print('uplaode 12',elapsed_time)

    #         if elapsed_time < 60:
    #             # If uploads exceed the limit in less than a minute, wait for the remaining time
    #             wait_time = 60 - elapsed_time
    #             print(f'Upload limit reached. Waiting for {wait_time:.2f} seconds...')
    #             time.sleep(wait_time)
    #             uploads = 0  # Reset the counter and start a new minute
    #             start_time = time.time()
        ###

        files = [('file', ('file', open(file, 'rb'), 'application/octet-stream'))]
        headers = {'x-api-key': API_Key}
        response = requests.post('https://api.chatpdf.com/v1/sources/add-file', headers=headers, files=files)
        # Update the upload counter
        #uploads = uploads + 1
        ## chatpdf jargon; uploading papers
        if response.status_code == 200:
            sID = response.json()['sourceId']
        else:
            print('Status:', response.status_code)
            print('Error:', response.text)
            continue


        ########################## 'Read' papers in chatpdf ##########################
        headers = {
            'x-api-key': API_Key,
            "Content-Type": "application/json"
        }
        # the defualt question isnt really working with current dataframe setup
        if question == 'default':
            question = f'provide a quote from the text about how {hazard} impacts {food}?'
        
        # more chatpdf jargon, input question:
        data = {
            'sourceId': sID,
            'messages': [
                # set up few-shot prompting (i'm not entirely sure if we should have our same questions before this to show it how to answer or if this is sufficient)
                # code from: https://www.chatpdf.com/docs/api/backend
                {
                    'role': "assistant",
                    'content': '1a) "Understanding the responses of animals to acute heat stress can help to reveal and predict the effect of more frequent extreme hot weather episodes on animal populations and ecosystems in the content of global climate change". 2a) Indirect physiology 3a) Negative 4a) The warming negatively impacts Chinese softshell turtle through indirect physiological mechanisms. 5a) "The results suggest that the Chinese soft-shelled turtle has a strong capacity to activate its antioxidant defense system to cope with acute heat stress". 6a) Beijing, China. 7a) The impact of warming on Chinese softshell turtle via indirect physiology is negative. The study was in Beijing, China.'
                },
                # actual question
                {
                    'role': "user",
                    'content': question,
                }
            ]
        }
        response = requests.post(
            'https://api.chatpdf.com/v1/chats/message', headers=headers, json=data)

        # if results are returned
        if response.status_code == 200:
            #print('result:', response.json()['content'])
            results = response.json()['content']
            segments = results.split('a)')  # Split the text using the question format ('a)')
            #segments = results.split('\n')  # could also split it by lines - might work well if a) is bad
            print(segments)
            print(len(segments))
            # split answer into small segments to put in dataframe
            ### segments for a. 
            if len(segments) ==8:
                gen_quote = segments[1].strip()[:-3]
                impact = segments[2].strip()[:-3]
                direction = segments[3].strip()[:-3]
                mechanism = segments[4].strip()[:-3]
                quote = segments[5].strip()[:-3]
                location = segments[6].strip()[:-3]
                full_answer = segments[7].strip()[:-3]

            else:
                gen_quote = 'Nan'
                impact = 'Nan'
                direction = 'Nan'
                mechanism = 'Nan'
                quote = 'Nan'
                location = 'Nan'
                full_answer = 'Nan'
            
            print(full_answer)
            # Create a row dictionary for this PDF
            row_data = {
                'filename': os.path.basename(file),
                'food': food,
                'hazard': hazard,
                'gen_quote': gen_quote,
                'impact': impact,
                'direction': direction,
                'mechanism': mechanism,
                'quote': quote,
                'location': location,
                'full_answer': full_answer
            }

            # Append the row data to the results
            all_results = pd.concat([all_results, pd.DataFrame([row_data])], ignore_index=True)

        else: #if it's baaad
            print('Status:', response.status_code)
            print('Error:', response.text)
    # Return dataframe for one pdf
    return all_results 